# Searching for optimal locations for Cloud Kitchens in Mumbai

Project is divided into 3 stages,
- A. Generating Datasets
- B. Search Space Creation
- C. Optimal Location and Location Set Identification
- D. Cuisine recommendations for best locations

###### Approach:
Our analysis is based on computing the distance betweent to location coordinates, and optimising the search using this are the key parameter.

Distance is defined as the sum of the x(or W-E) and y (or N-S) intercept.

The analysis progresses with first rating each venue location as follows,
    rating = (minimum service time for a given set of city centres)/(service time of a location for given set of city centres)

This data will be used to train a model that can generate similar ratings for any point using the following three attributes,
- Number of city centres within serviceable distance
- Average distance between location and the city centres
- Average potential serviceable population

Then we generate a search space of points using the city centre location coordinates, and find points with the best ratings.

It is assumed that these points represent locations that have the maximum potential serviceable population for given set of city centres possible.

After that top locations are sorted and searched for a combination of best locations that serve the maximum proportion of population of the city centres.

Once we have obtained the list of best locations, we simply check the venue categories available at all city centres near our best locations.

We use the Foursquare location data to compute the average venue category at each city centre, and then use the population of the city centres to compute the weighted average for a best location.

###### Key comments
- We have two datasets for city centers, but one lacks population data, hence it is automatically dropped from further analysis
- Still we have utilised that data to generate venue locations from the Foursquare API as it provides more results
- Venue category or cuisine reccomendations are based Foursquare location data
- Functions for all computations will be available in related notebooks



## C. Optimal Location and Location Set Identification

### C.1. Importing libraries and datasets

In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
map_point_score_list = pd.read_csv('map_point_score_list.csv').to_numpy()

In [3]:
len(map_point_score_list)

24347

In [4]:
# map_points with rating greater than 1, this forms about 1.18% of the total map_points set

count = 0

for map_point in map_point_score_list:
    
    if float(map_point[2].replace("[","").replace("]","")) > 1:
        count += 1
count

317

In [5]:
#map_point_score_list[0]

In [6]:
#map_point_score_list[0][3].replace('[','').replace(']','').split(",")[0]

In [7]:
#list1 = map_point_score_list[0][3].replace("[","").replace("]","").split(",")
#float(list1[2])

In [8]:
city_centers = pd.read_csv('m_ward_data.csv').to_numpy()
city_centers[:10]

array([['A', 'Colaba', 13, '43,661', '2,10,847', '16,868', 'A', 400001,
        18.94959357, 72.83815173, True],
       ['B', 'Sanhurst Road', 3, '27,225', '1,40,633', '56,253', 'B',
        400009, 18.95702047, 72.84200447, True],
       ['C', 'Marine Lines', 2, '39,657', '2,02,922', '1,12,734', 'C',
        400002, 18.94638502, 72.8252676, True],
       ['D', 'Grant Road', 7, '79,131', '3,82,841', '58,006', 'D',
        400007, 18.9584585, 72.81496344, True],
       ['E', 'Byculla', 7, '80,970', '4,40,335', '59,505', 'E', 400008,
        18.96943916, 72.82582256, True],
       ['F South', 'Parel', 14, '80,777', '3,96,122', '28,294',
        'F South', 400012, 19.0008, 72.83085, True],
       ['F North', 'Matunga', 13, '1,12,765', '5,24,393', '40,338',
        'F North', 400019, 19.028744, 72.8441474, True],
       ['G South', 'Elphinstone', 10, '92,525', '4,57,931', '45,793',
        'G South', 400018, 19.0166742, 72.8166592, True],
       ['G North', 'Dadar/Plaza', 9, '1,20,643', '5

### C.2. Function definitions

In [9]:
# function to compute distance between two points

# assuming most streets are parallel to either north-south axis or east-west axis
# thus distance is computed as sum of absolute distances

# scale used is 1 degree difference is equal to 111 KM in either axis

def distance(cord1, cord2):
    
    x = (cord1[0]-cord2[0])*111
    
    y = (cord1[1]-cord2[1])*111
    
    if x*y > 0:
        if x > 0:
            return x + y
        else:
            return -1*(x + y)
    else:
        if x > 0:
            return x + (-1 * y)
        else:
            return (-1*x) + y

In [10]:
# function to create a set of top rated locations

# sorting the locations groups (based on rating), then city center count, and then average population served by the location

def best_locations_set_gen(map_point_score_list, rating_set, check_sets=0):
    
    best_locations_set = []
    
    for rating in rating_set:
        best_locations_set.append([])

    for map_point in map_point_score_list:

        for count, rating in enumerate(rating_set):
            
            point_rating = float(map_point[2].replace('[','').replace(']',''))

            if point_rating >= rating and point_rating < 1:

                best_locations_set[count].append(map_point)

                break
                
    for i in range(len(best_locations_set)):
        print("Size of Set {}: {}".format(i,len(best_locations_set[i])))
    
    
    # sorting best locations based on total population served -> product of city centre count and average population
    
    for count, best_locations in enumerate(best_locations_set):
        
        if count >= check_sets:
            return best_locations_set
        
        for i in range(len(best_locations)):
            
            loc_details1 = best_locations[i][3].replace("[","").replace("]","").split(",")
            
            total_pop1 = float(loc_details1[0])*float(loc_details1[2])
            
            for j in range(i+1, len(best_locations)):
                
                loc_details2 = best_locations[j][3].replace("[","").replace("]","").split(",")
            
                total_pop2 = float(loc_details2[0])*float(loc_details2[2])
                
                if total_pop2 >= total_pop1 :
                    
                    sort_set = best_locations[i]
                    
                    best_locations[i] = best_locations[j]
                    
                    best_locations[j] = sort_set
    
        print("Set {} has been sorted".format(count))
    
    return best_locations_set

In [11]:
# function to generate best_location city centre wise population served data set
# check_groups determines how many rating groups are searched for best locations

def best_location_city_centre_population_set(best_locations_set, city_centers, distance_limit = 15, initial_distance_limit = 2.5, check_groups = 2):

    population_set = []

    # city center wise population list
    for city_centre in city_centers:

        population_set.append(int(city_centre[4].replace(',','')))

    # searching for combination of best locations that address the entire population
    map_point_population_set = []

    for best_locations in best_locations_set:

        for map_point in best_locations:

            map_point_population = [map_point,1]
            
            map_point_cords = map_point[1].replace('[','').replace(']','').split(',')
            
            map_point_cords = [float(map_point_cords[0]),float(map_point_cords[1])]

            for city_centre in city_centers: 
                
                city_centre_cords = [float(city_centre[8]),float(city_centre[9])]
                
                city_centre_distance = distance(map_point_cords, city_centre_cords)

                if city_centre_distance <= (distance_limit-initial_distance_limit) and city_centre_distance > 0:

                    metric = distance_limit - city_centre_distance - initial_distance_limit

                    max_d = ((city_centre[2]/math.pi)**(1/2))
            
                    if metric > max_d:
                        metric = max_d

                    city_centre_population = float(city_centre[4].replace(",",""))

                    c1 = 2/(math.pi*(max_d**2))
                    c2 = -2/(math.pi*(max_d**4))
                    
                    population_addressed = int(city_centre_population*((math.pi)*((c1*(metric**2))+((c2/2)*(metric**4)))))
                    
                    #cc_count = int(map_point[3].replace('[','').replace(']','').split(",")[0])
                    
                    map_point_population.append(population_addressed)
                    
                else:
                    map_point_population.append(0.0)
                
            
            map_point_population_set.append(map_point_population)
        
        # terminate search for best locations after check_groups is done
        if check_groups == 0:
            break
        else:
            check_groups -= 1
    
    return map_point_population_set, population_set

In [12]:
# function to sort the datasets based on population

def map_point_population_set_sorter(map_point_population_set, population_set):
    
    tracker = np.arange(0,len(population_set),1)
    
    # sorting the population based on number of best locations available available in ascending order
    
    # counting number of locations serving a city centre
    counter = np.zeros(len(population_set))

    for i in range(len(map_point_population_set)):

        for j in range(2,len(map_point_population_set[i])):

            if map_point_population_set[i][j] > 0:
                counter[j-2] += 1
    
       
    for i in range(len(counter)):
        
        for j in range(i+1,len(counter)):
            
            if counter[i] > counter[j]:
                
                sort_set = counter[i]
                
                counter[i] = counter[j]
                
                counter[j] = sort_set
                
                tracker_sort = tracker[i]
                
                tracker[i] = tracker[j]
                
                tracker[j] = tracker_sort
       
    # tracker based on sorting done above
    print("Tracker : {}".format(tracker))
    
    #map_point_population_set_sorted = []
    #population_set_sorted = []
    
    # building sorted population set
    #for i in range(len(tracker)):
     #   population_set_sorted.append(population_set[tracker[i]])       
        
    # building sorted map_point_population_set
    #for index in range(len(map_point_population_set)):
        
        # build sorted set
     #   sorted_data = [map_point_population_set[index][0],1]
        
      #  for i in range(len(tracker)):
       #     sorted_data.append(map_point_population_set[index][tracker[i]+2])        
        
        #map_point_population_set_sorted.append(sorted_data)
    
    # print overview
    for count, sol_count in enumerate(counter):
        print("For City Center {} with population {} has {} locations are available".format((tracker[count]),population_set[tracker[count]],(sol_count)))
    
    return tracker #map_point_population_set_sorted, population_set_sorted

In [13]:
# function to  find out best set of best locations!

# city center population set is sorted based on number of locations available

# we search for map points that service each city center in that order

# if a city centre is fully serviced, then all map points that have a positive contribution to the city centre are dropped from further search

def best_locations_search(func_map_point_population_set, func_population_set, tracker, serviceable_population = 1.0):

    best_locations = []
    
    population_set = np.copy(func_population_set)
    map_point_population_set = np.copy(func_map_point_population_set)
    
    # population list is sorted by size
    for i in tracker:
        
        search_space_size = 0
        
        for map_point in map_point_population_set:
            
            if map_point[1] == 1:
                search_space_size += 1
        
        print("For City Center: {} out of {} with Search Space Size {}".format((i+1),len(population_set), search_space_size))
        
        print("City Centre Population start: {}".format(population_set[i]))
        
        for j in range(len(map_point_population_set)):
            
            # checking for available population at CC, positive contribution by map point to city centre, and positive status of map point
            if population_set[i] > 0 and map_point_population_set[j][i+2] > 0 and map_point_population_set[j][1] == 1:
                
                #tot_pop = 0
                
                #for count in range(2,len(map_point_population_set[j])):
                    
                    #tot_pop += map_point_population_set[j][count]
                
                # updating the population of all city centres
                for k in tracker:
                    population_set[k] = population_set[k] - (map_point_population_set[j][k+2]*serviceable_population)
                
                # adding location to best location set along with population contributions
                best_locations.append([j, map_point_population_set[j]])
                
                # updating map point status to selected
                map_point_population_set[j][1] = 2
                
                print("Total Location Count: {}, new point number: {}".format(len(best_locations),j))
            
                # removing locations that may overserve any city centre
                count = 0
                
                for k in tracker:
                    
                    if population_set[k] <= 1:

                        for l in range(len(map_point_population_set)):

                            if map_point_population_set[l][k+2] > 0 and map_point_population_set[l][1] == 1:
                                map_point_population_set[l][1] = 0
                                count += 1
                            
                print("Deleted {} points from further search for {}".format(count,k))
                
        print("City Centre Population end: {}".format(population_set[i]))

    return best_locations, population_set

In [14]:
# function to generate n lists of best locations

def best_locations_solutions_gen(map_point_score_list, 
                                 city_centers, 
                                 distance_limit = 15, 
                                 initial_distance_limit = 2.5, 
                                 serviceable_population = 1.0, 
                                 rating_set = [1,0.5,0], 
                                 check_sets=1, 
                                 check_groups = 3):
    
    
    # creating rating groups
    best_locations_set = best_locations_set_gen(map_point_score_list, rating_set, check_sets)
    
    # generating population list for best locations
    map_point_population_set, population_set = best_location_city_centre_population_set(best_locations_set, 
                                                                                        city_centers, 
                                                                                        distance_limit, 
                                                                                        initial_distance_limit, 
                                                                                        check_groups)
    
    # sorting sets based on population
    tracker = map_point_population_set_sorter(map_point_population_set, population_set)
    
    # generating list of best solutions
    best_locations_solutions = []
    
    for count in range(len(tracker)):
        
        loop_tracker = []
        
        print("Run: {}".format(count+1))
        
        for i in range(len(tracker)):
            
            if i > count:
                loop_tracker.append(tracker[i])
            else:
                loop_tracker.insert(0,tracker[i])
        
        location_list, new_population_set = best_locations_search(map_point_population_set, 
                                                                  population_set, 
                                                                  loop_tracker, 
                                                                  serviceable_population)
        
        best_locations_solutions.append([location_list, new_population_set])
    
    
    return best_locations_solutions

In [15]:
# function to return best location solution

def best_venue_locations_gen(best_locations_solutions):
    
    unserved_population = []

    for count, best_locations in enumerate(best_locations_solutions):

        pop = 0

        for i in range(len(best_locations[1])):

            if best_locations[1][i] > 0:
                pop += best_locations[1][i]

        unserved_population.append(pop)

        print("Solution {} fails to served {} despite venue count of {}".format(count, pop, len(best_locations[0])))
        
    best_sol = unserved_population[0]
    best_solID = 0
    
    for i in range(len(unserved_population)):
        
        if unserved_population[i] < best_sol:
            best_sol = unserved_population[i]
            best_solID = i
    
    print("Best solution: {}".format(best_solID))
    
    return best_locations_solutions[best_solID]

### C.3. Searching for best locations

Creating set of rated locations. We can limit the sorting if the size of the map_point_score_list is too large.

We can created rating groups starting from 0.95 to 0, and as mentioned above, any rating greater than 1 is ignored.

In [16]:
rating_set = np.arange(0.95,0,-.05)
rating_set

array([0.95, 0.9 , 0.85, 0.8 , 0.75, 0.7 , 0.65, 0.6 , 0.55, 0.5 , 0.45,
       0.4 , 0.35, 0.3 , 0.25, 0.2 , 0.15, 0.1 , 0.05])

In [17]:
#best_locations_set = best_locations_set_gen(map_point_score_list, rating_set, check_sets=1)

In [18]:
#len(best_locations_set[0])

In [19]:
#best_locations_set[0][0]

In [20]:
#map_point_population_set, population_set = best_location_city_centre_population_set(best_locations_set, 
#                                                                                    city_centers, 
#                                                                                    distance_limit = 15, 
#                                                                                    initial_distance_limit = 2.5, 
#                                                                                    check_groups = 3)

In [21]:
#len(map_point_population_set)

In [22]:
#tracker = map_point_population_set_sorter(map_point_population_set, population_set)

We create a set of trackers that we use to prioritise a city centre; we use number of available solutions as a metric

In [23]:
#tracker_set = []

#for count in range(len(tracker)):
        
#    loop_tracker = []

#    for i in range(len(tracker)):

#        if i > count:
#            loop_tracker.append(tracker[i])
#        else:
#            loop_tracker.insert(0,tracker[i])
    
#    tracker_set.append(loop_tracker)
#    print(loop_tracker)

In [24]:
#sol1, new_pop_set = best_locations_search(map_point_population_set, 
#                                          population_set, 
#                                          tracker_set[23], 
#                                          serviceable_population = 0.7)

In [25]:
#len(sol1)

Assuming that a location can service only 70% of a city center's population, generate a list of best locations.

In [26]:
serviceable_population = 0.70

# generating best locations list
best_locations_solutions = best_locations_solutions_gen(map_point_score_list, 
                                                        city_centers, 
                                                        distance_limit = 15, 
                                                        initial_distance_limit = 2.5, 
                                                        rating_set = rating_set, 
                                                        check_sets = 5, 
                                                        check_groups = 5, 
                                                        serviceable_population = serviceable_population)

Size of Set 0: 4832
Size of Set 1: 4781
Size of Set 2: 7273
Size of Set 3: 3617
Size of Set 4: 1616
Size of Set 5: 1778
Size of Set 6: 128
Size of Set 7: 1
Size of Set 8: 2
Size of Set 9: 0
Size of Set 10: 0
Size of Set 11: 0
Size of Set 12: 0
Size of Set 13: 0
Size of Set 14: 0
Size of Set 15: 0
Size of Set 16: 0
Size of Set 17: 0
Size of Set 18: 0
Set 0 has been sorted
Set 1 has been sorted
Set 2 has been sorted
Set 3 has been sorted
Set 4 has been sorted
Tracker : [21  2  3  0  4 23  1 18 20 19  5  7 22  6 15 10 17  8 14  9 16 11 13 12]
For City Center 21 with population 363827 has 1.0 locations are available
For City Center 2 with population 202922 has 1050.0 locations are available
For City Center 3 with population 382841 has 1168.0 locations are available
For City Center 0 with population 210847 has 2403.0 locations are available
For City Center 4 with population 440335 has 2697.0 locations are available
For City Center 23 with population 330195 has 2809.0 locations are available

C:\Programming\lib\site-packages\numpy\lib\function_base.py:792: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, order=order, subok=subok, copy=True)


Deleted 0 points from further search for 12
Total Location Count: 5, new point number: 4832
Deleted 2081 points from further search for 12
City Centre Population end: -78450
For City Center: 2 out of 24 with Search Space Size 0
City Centre Population start: -56252
City Centre Population end: -56252
For City Center: 19 out of 24 with Search Space Size 0
City Centre Population start: -318708
City Centre Population end: -318708
For City Center: 21 out of 24 with Search Space Size 0
City Centre Population start: 153923
City Centre Population end: 153923
For City Center: 20 out of 24 with Search Space Size 0
City Centre Population start: -235955
City Centre Population end: -235955
For City Center: 6 out of 24 with Search Space Size 0
City Centre Population start: -158449
City Centre Population end: -158449
For City Center: 8 out of 24 with Search Space Size 0
City Centre Population start: -183172
City Centre Population end: -183172
For City Center: 23 out of 24 with Search Space Size 0
City

For City Center: 1 out of 24 with Search Space Size 23897
City Centre Population start: 210847
Total Location Count: 1, new point number: 4702
Deleted 0 points from further search for 12
Total Location Count: 2, new point number: 4703
Deleted 21811 points from further search for 12
City Centre Population end: -84338
For City Center: 4 out of 24 with Search Space Size 2084
City Centre Population start: -153136
City Centre Population end: -153136
For City Center: 3 out of 24 with Search Space Size 2084
City Centre Population start: -81169
City Centre Population end: -81169
For City Center: 22 out of 24 with Search Space Size 2084
City Centre Population start: 363827
Total Location Count: 3, new point number: 22122
Deleted 0 points from further search for 12
City Centre Population end: 312135
For City Center: 5 out of 24 with Search Space Size 2083
City Centre Population start: -176134
City Centre Population end: -176134
For City Center: 24 out of 24 with Search Space Size 2083
City Centr

City Centre Population end: 185866
For City Center: 12 out of 24 with Search Space Size 0
City Centre Population start: -463778
City Centre Population end: -463778
For City Center: 14 out of 24 with Search Space Size 0
City Centre Population start: 196382
City Centre Population end: 196382
For City Center: 13 out of 24 with Search Space Size 0
City Centre Population start: -280271
City Centre Population end: -280271
Run: 7
For City Center: 2 out of 24 with Search Space Size 23897
City Centre Population start: 140633
Total Location Count: 1, new point number: 2507
Deleted 0 points from further search for 12
Total Location Count: 2, new point number: 2508
Deleted 23893 points from further search for 12
City Centre Population end: 131582
For City Center: 24 out of 24 with Search Space Size 2
City Centre Population start: 330195
Total Location Count: 3, new point number: 20502
Deleted 1 points from further search for 12
City Centre Population end: 99058
For City Center: 5 out of 24 with Se

...

Deleted 23893 points from further search for 12
City Centre Population end: -166010
For City Center: 8 out of 24 with Search Space Size 2
City Centre Population start: 457931
Total Location Count: 3, new point number: 9612
Deleted 0 points from further search for 12
Total Location Count: 4, new point number: 20500
Deleted 0 points from further search for 12
City Centre Population end: -183172
For City Center: 6 out of 24 with Search Space Size 0
City Centre Population start: -158449
City Centre Population end: -158449
For City Center: 20 out of 24 with Search Space Size 0
City Centre Population start: 589886
City Centre Population end: 589886
For City Center: 21 out of 24 with Search Space Size 0
City Centre Population start: 513077
City Centre Population end: 513077
For City Center: 19 out of 24 with Search Space Size 0
City Centre Population start: 796775
City Centre Population end: 796775
For City Center: 2 out of 24 with Search Space Size 0
City Centre Population start: -56252
City

For City Center: 11 out of 24 with Search Space Size 23897
City Centre Population start: 337391
Total Location Count: 1, new point number: 0
Deleted 0 points from further search for 12
Total Location Count: 2, new point number: 1
Deleted 23895 points from further search for 12
City Centre Population end: -134954
For City Center: 16 out of 24 with Search Space Size 0
City Centre Population start: -165616
City Centre Population end: -165616
For City Center: 7 out of 24 with Search Space Size 0
City Centre Population start: -209756
City Centre Population end: -209756
For City Center: 23 out of 24 with Search Space Size 0
City Centre Population start: 691227
City Centre Population end: 691227
For City Center: 8 out of 24 with Search Space Size 0
City Centre Population start: 457931
City Centre Population end: 457931
For City Center: 6 out of 24 with Search Space Size 0
City Centre Population start: 396122
City Centre Population end: 396122
For City Center: 20 out of 24 with Search Space Si

City Centre Population end: -280271
Run: 19
For City Center: 15 out of 24 with Search Space Size 23897
City Centre Population start: 674850
Total Location Count: 1, new point number: 0
Deleted 0 points from further search for 12
Total Location Count: 2, new point number: 1
Deleted 23895 points from further search for 12
City Centre Population end: -269939
For City Center: 9 out of 24 with Search Space Size 0
City Centre Population start: -232802
City Centre Population end: -232802
For City Center: 18 out of 24 with Search Space Size 0
City Centre Population start: -175138
City Centre Population end: -175138
For City Center: 11 out of 24 with Search Space Size 0
City Centre Population start: -134954
City Centre Population end: -134954
For City Center: 16 out of 24 with Search Space Size 0
City Centre Population start: -165616
City Centre Population end: -165616
For City Center: 7 out of 24 with Search Space Size 0
City Centre Population start: -209756
City Centre Population end: -209756

City Centre Population end: 202922
For City Center: 22 out of 24 with Search Space Size 0
City Centre Population start: 363827
City Centre Population end: 363827
For City Center: 12 out of 24 with Search Space Size 0
City Centre Population start: -324001
City Centre Population end: -324001
For City Center: 14 out of 24 with Search Space Size 0
City Centre Population start: -311287
City Centre Population end: -311287
For City Center: 13 out of 24 with Search Space Size 0
City Centre Population start: -280271
City Centre Population end: -280271
Run: 22
For City Center: 12 out of 24 with Search Space Size 23897
City Centre Population start: 810002
Total Location Count: 1, new point number: 0
Deleted 0 points from further search for 12
Total Location Count: 2, new point number: 1
Deleted 23895 points from further search for 12
City Centre Population end: -324001
For City Center: 17 out of 24 with Search Space Size 0
City Centre Population start: -247823
City Centre Population end: -247823


City Centre Population end: 589886
For City Center: 21 out of 24 with Search Space Size 0
City Centre Population start: 513077
City Centre Population end: 513077
For City Center: 19 out of 24 with Search Space Size 0
City Centre Population start: 796775
City Centre Population end: 796775
For City Center: 2 out of 24 with Search Space Size 0
City Centre Population start: 140633
City Centre Population end: 140633
For City Center: 24 out of 24 with Search Space Size 0
City Centre Population start: 330195
City Centre Population end: 330195
For City Center: 5 out of 24 with Search Space Size 0
City Centre Population start: 440335
City Centre Population end: 440335
For City Center: 1 out of 24 with Search Space Size 0
City Centre Population start: 210847
City Centre Population end: 210847
For City Center: 4 out of 24 with Search Space Size 0
City Centre Population start: 382841
City Centre Population end: 382841
For City Center: 3 out of 24 with Search Space Size 0
City Centre Population sta

Selecting a set of best locations that has minimum unserved population

In [27]:
best_venue_locations = best_venue_locations_gen(best_locations_solutions)

Solution 0 fails to served 3238644 despite venue count of 5
Solution 1 fails to served 3238644 despite venue count of 5
Solution 2 fails to served 3238644 despite venue count of 5
Solution 3 fails to served 3238644 despite venue count of 5
Solution 4 fails to served 3942087 despite venue count of 3
Solution 5 fails to served 2493974 despite venue count of 4
Solution 6 fails to served 4123407 despite venue count of 3
Solution 7 fails to served 2815362 despite venue count of 4
Solution 8 fails to served 2815362 despite venue count of 4
Solution 9 fails to served 5642867 despite venue count of 2
Solution 10 fails to served 5463313 despite venue count of 2
Solution 11 fails to served 5509123 despite venue count of 2
Solution 12 fails to served 2997735 despite venue count of 4
Solution 13 fails to served 5516618 despite venue count of 2
Solution 14 fails to served 5516618 despite venue count of 2
Solution 15 fails to served 5516618 despite venue count of 2
Solution 16 fails to served 551661

This provides us with the minimum of number of top-rated locations a business would need to set up cloud kitchens at, to service the maximum proportion of the population of all city centres.

In [28]:
len(best_venue_locations[0])

4

In [29]:
#best_venue_locations[0]

Generating a set with the location coordinates of the set of best locations. This will be used to generate the Venue Category/Cuisine reccommendations.

In [30]:
best_locations_cords_set = []

for best_location in best_venue_locations[0]:
    
    cord_set = []
   
    cords = best_location[1][0][1].replace("[","").replace("]","").split(',')

    cord_set.append([float(cords[0]),float(cords[1])])
    
    best_locations_cords_set.append(cord_set)

best_locations_cords_set

[[[19.1373666, 72.9442834]],
 [[19.174673866666666, 72.86115685666665]],
 [[19.055614976666664, 72.84901185777778]],
 [[19.002550423333332, 72.83014521222223]]]

Storing map point score list for further analysis

In [31]:
pd.DataFrame(best_venue_locations).to_csv('best_venue_locations.csv')

In [32]:
pd.DataFrame(best_locations_cords_set).to_csv('best_locations_cords_set.csv')

### C.4 Analysis the set of best locations generated

In [33]:
best_venue_locations

[[[4831,
   array([array([22, '[19.1373666, 72.9442834]', '[0.97789291]',
          '[[3, 8.055100910000348, 605016.6666666667]]'], dtype=object),
          2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 199681,
          0.0, 0.0, 674850, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 330195],
         dtype=object)],
  [4832,
   array([array([24210, '[19.174673866666666, 72.86115685666665]', '[0.9049411]',
          '[[9, 7.118691014444468, 625726.7777777778]]'], dtype=object),
          2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 810002,
          700680, 52975, 674850, 0.0, 0.0, 437848, 796774, 589886, 513077,
          0.0, 0.0, 253583], dtype=object)],
  [2539,
   array([array([9762, '[19.055614976666664, 72.84901185777778]', '[0.9572984]',
          '[[14, 6.599315242857111, 515714.7142857143]]'], dtype=object),
          2, 0.0, 121810, 0.0, 0.0, 49860, 396122, 524392, 457931, 582007,
          580835, 337390, 810002, 700680, 778218, 0.0, 414040, 619556,
     

In [34]:
unserved_population = 0
total_population = 11976439

for pop in best_venue_locations[1]:
    
    if pop > 0:
        
        unserved_population += pop

print("{}% of the total population is unserved.".format((unserved_population*100/total_population)))

20.82400286095057% of the total population is unserved.


In [35]:
city_centers_unserved = 0
cc_ID = []
cc_pop = 0

# city center wise population list

population_set = []

for city_centre in city_centers:

    population_set.append(int(city_centre[4].replace(',','')))


for i in range(len(population_set)):
    if population_set[i] == best_venue_locations[1][i]:
        city_centers_unserved += 1
        cc_ID.append(i)
        cc_pop += population_set[i]

print("Count: {} with total population of {}, IDs: {}".format(city_centers_unserved,cc_pop,cc_ID))

city_centers[21]

Count: 2 with total population of 1055054, IDs: [21, 22]


array(['R North', 'Dahiser', 18, '83,433', '3,63,827', '20,213',
       'R North', 400068, 19.3344948, 72.8654778, True], dtype=object)

In [36]:
# function to compute the average service time

def ast_gen(city_centers, best_locations_cords_set, distance_limit = 15, initial_distance_limit = 2.5, for_cc = True):
    
    ast_list = []
    
    # extracting city center details
    cc_details = []
    for city_centre in city_centers:

            city_centre_cords = [city_centre[8],city_centre[9]]
            max_d = ((city_centre[2]/math.pi)**(1/2))
            
            cc_details.append([city_centre_cords,max_d])

    if for_cc:
        set1 = cc_details
        set2 = best_locations_cords_set
    else:
        set2 = cc_details
        set1 = best_locations_cords_set
    
    for i in range(len(set1)):
        
        avg_ast = 0.0
        count = 0
        avg_dist = 0
        
        for j in range(len(set2)):
            
            if for_cc:
                city_centre_cords = set1[i][0]
                location = set2[j][0]
            else:
                city_centre_cords = set2[j][0]
                location = set1[i][0]
            
            city_centre_distance = distance(location, city_centre_cords)

            if city_centre_distance > 0 and city_centre_distance <= distance_limit:

                metric = distance_limit - city_centre_distance - initial_distance_limit

                if for_cc:
                    max_d = set1[i][1]
                else:
                    max_d = set2[i][1]

                if metric > max_d:
                    metric = max_d

                c1 = 2/(math.pi*(max_d**2))
                c2 = -2/(math.pi*(max_d**4))
                c3 = city_centre_distance + initial_distance_limit

                # we compute the average service time for the population
                ast = (((math.pi/10)*(((c1/3)*(metric**3))+((c2/5)*(metric**5))))+(c3/20))

                avg_ast += ast
                avg_dist += city_centre_distance
                count += 1

        if count != 0:
            ast_list.append([avg_ast/count,avg_dist/count])
        else:
            ast_list.append([0,0])
        
    return ast_list

In [37]:
cc_average_time = ast_gen(city_centers, best_locations_cords_set, distance_limit = 15, initial_distance_limit = 2.5, for_cc = True)   
print(cc_average_time)

average_service_time,average_distance, count = 0.0,0,0

for i in range(len(cc_average_time)):
    if cc_average_time[i][0] > 0:
        average_service_time += cc_average_time[i][0]
        count += 1
        average_distance += cc_average_time[i][1]

print(average_service_time/count,average_distance/count)

[[0.6448133201948918, 9.870392258332581], [0.6005204995326641, 9.046006358332727], [2.9798705702092234, 10.7679562650002], [0.6927930598922765, 10.571561745000869], [0.5529462503475132, 8.147346165000021], [0.3906173696918892, 4.1864785283325165], [0.3788547961850177, 3.9921815383328276], [0.44703308784742884, 5.489128895000432], [0.37351115810048197, 4.06751982833323], [0.5372271536774115, 7.7639863488881], [0.37672667818628613, 3.9921815383328276], [0.6118984317994932, 8.679618688888652], [0.6752174923779525, 10.553632161666304], [0.6981533567003101, 11.395527509999777], [0.6195356910880849, 8.740189045556306], [0.6347610083519686, 9.520507924999405], [0.7204783899670002, 11.798443634999938], [0.6200221874829305, 9.329718144444172], [0.5181331622914416, 6.551065709997914], [0.45675806936418006, 5.358546089999017], [0.5694088936071462, 6.760485710001948], [0, 0], [0.7913139272315604, 13.368152910001747], [0.5695119441745955, 7.2322625350012935]]
0.672178543404424 8.138386501497513


In [38]:
bl_average_time = ast_gen(city_centers, best_locations_cords_set, distance_limit = 15, initial_distance_limit = 2.5, for_cc = False)   
print(cc_average_time)

average_service_time,average_distance, count = 0.0,0,0

for i in range(len(bl_average_time)):
    if cc_average_time[i][0] > 0:
        average_service_time += bl_average_time[i][0]
        count += 1
        average_distance += bl_average_time[i][1]

print(average_service_time/count,average_distance/count)

[[0.6448133201948918, 9.870392258332581], [0.6005204995326641, 9.046006358332727], [2.9798705702092234, 10.7679562650002], [0.6927930598922765, 10.571561745000869], [0.5529462503475132, 8.147346165000021], [0.3906173696918892, 4.1864785283325165], [0.3788547961850177, 3.9921815383328276], [0.44703308784742884, 5.489128895000432], [0.37351115810048197, 4.06751982833323], [0.5372271536774115, 7.7639863488881], [0.37672667818628613, 3.9921815383328276], [0.6118984317994932, 8.679618688888652], [0.6752174923779525, 10.553632161666304], [0.6981533567003101, 11.395527509999777], [0.6195356910880849, 8.740189045556306], [0.6347610083519686, 9.520507924999405], [0.7204783899670002, 11.798443634999938], [0.6200221874829305, 9.329718144444172], [0.5181331622914416, 6.551065709997914], [0.45675806936418006, 5.358546089999017], [0.5694088936071462, 6.760485710001948], [0, 0], [0.7913139272315604, 13.368152910001747], [0.5695119441745955, 7.2322625350012935]]
0.6955114159642888 8.894409617453675


### C.5. Displaying best locations

In [39]:
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [40]:
city_centers_pd = pd.read_csv('m_ward_data.csv')

In [41]:
# creating new feature group for city centers

def plot_city_centers(city_centers, _map):
    
    city_centers_plot = folium.map.FeatureGroup()

    print(city_centers.shape)

    # plotting venues
    for lat, lng, in zip(city_centers.Lat.astype(float), city_centers.Long.astype(float)):
        city_centers_plot.add_child(
            folium.features.CircleMarker(
                [lat, lng],
                radius=3, # define how big you want the circle markers to be
                color='blue',
                fill=True,
                fill_color='blue',
                fill_opacity=0.2
            )
        )

    # add venues to map
    _map.add_child(city_centers_plot)
    
    return

In [42]:
# creating new feature group for venues

def plot_best_locations(venue_data, _map):
    
    venues = folium.map.FeatureGroup()

    print(venue_data.shape)

    # plotting venues
    for lat, lng, in zip(venue_data['Lat'], venue_data['Long']):
        venues.add_child(
            folium.features.CircleMarker(
                [lat, lng],
                radius=10, # define how big you want the circle markers to be
                color='red',
                fill=False,
                fill_color='red',
                fill_opacity=0.0
            )
        )

    # add venues to map
    _map.add_child(venues)
    
    return

In [43]:
address = 'Mumbai, India'

geolocator = Nominatim(user_agent="mu_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai City are 19.0759899, 72.8773928.


In [44]:
best_locations_map_points = pd.DataFrame(columns = ['Lat','Long'])

for map_point in best_locations_cords_set:
    
    lat = map_point[0][0]
    long = map_point[0][1]
    
    best_locations_map_points = best_locations_map_points.append({'Lat':lat,'Long':long}, ignore_index=True)

best_locations_map_points

,Lat,Long
0,19.137367,72.944283
1,19.174674,72.861157
2,19.055615,72.849012
3,19.002550,72.830145


In [45]:
# create map and display it
mumbai_map = folium.Map(location=[latitude, longitude], zoom_start=11)

plot_city_centers(city_centers_pd, mumbai_map)
plot_best_locations(best_locations_map_points, mumbai_map)

mumbai_map

(24, 11)
(4, 2)


In [46]:
best_locations_cords_set

[[[19.1373666, 72.9442834]],
 [[19.174673866666666, 72.86115685666665]],
 [[19.055614976666664, 72.84901185777778]],
 [[19.002550423333332, 72.83014521222223]]]

In [47]:
count, d = 0,0

for i in range(len(best_locations_cords_set)):
    for j in range(i+1, len(best_locations_cords_set)):
        d += distance(best_locations_cords_set[i][0],best_locations_cords_set[j][0])
        count += 1
        
d/count

17.624608037777026